In [ ]:
import base64
import json
from flask import Flask, jsonify, request
import numpy as np
import cv2
import matplotlib.pyplot as plt
import threading
from definition import openai_algo, remove_grabcut_bg, do_seg, translate_korean_to_english
import sqlite3
import os
import smtplib
from stability_sdk import client
import stability_sdk.interfaces.gooseai.generation.generation_pb2 as generation
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from segment_anything import sam_model_registry, SamPredictor
import io
import warnings
from PIL import Image
import torch

sam_checkpoint = "C:/Users/do150/Desktop/sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

app = Flask(__name__)

def stability_algorithm(image, sentence):
    os.environ['STABILITY_HOST'] = 'grpc.stability.ai:443'

    os.environ['STABILITY_KEY'] = 'sk-S205IclhTAtN6Hsx1w3M61BvClNPRl4P6QKOKRLmxZeDqoef'
    
    stability_api = client.StabilityInference(
        key=os.environ['STABILITY_KEY'],
        verbose=True,
        engine="stable-diffusion-xl-beta-v2-2-2",  
    )
    
    image = Image.fromarray(image)

    new_width = image.width - (image.width % 64)
    new_height = image.height - (image.height % 64)
    input_image = image.resize((new_width, new_height))

    answers2 = stability_api.generate(
        prompt=sentence,
        init_image=input_image,
        start_schedule=0.5,
        seed=123467458,
        steps=30,
        cfg_scale=7.0,
        width=512,
        height=512,
        sampler=generation.SAMPLER_K_DPMPP_2M
    )

    for resp in answers2:
        for artifact in resp.artifacts:
            if artifact.finish_reason == generation.FILTER:
                warnings.warn(
                    "Your request activated the API's safety filters and could not be processed."
                    "Please modify the prompt and try again.")
            if artifact.type == generation.ARTIFACT_IMAGE:
                global img2
                img2 = Image.open(io.BytesIO(artifact.binary))
                img2.save("C:/Users/do150/Desktop/result/image-img2img.png")
                
    return 1

def text2image(sentence):
    os.environ['STABILITY_HOST'] = 'grpc.stability.ai:443'
    os.environ['STABILITY_KEY'] = 'sk-S205IclhTAtN6Hsx1w3M61BvClNPRl4P6QKOKRLmxZeDqoef'
    
    stability_api = client.StabilityInference(
        key=os.environ['STABILITY_KEY'],
        verbose=True,
        engine="stable-diffusion-xl-1024-v1-0",
    )
    
    answers = stability_api.generate(
        prompt=sentence,
        seed=4253978046,
        steps=50,
        cfg_scale=8.0,
        width=1024,
        height=1024,
        samples=1,
        sampler=generation.SAMPLER_K_DPMPP_2M
    )

    for resp in answers:
        for artifact in resp.artifacts:
            if artifact.finish_reason == generation.FILTER:
                warnings.warn(
                    "Your request activated the API's safety filters and could not be processed."
                    "Please modify the prompt and try again.")
            if artifact.type == generation.ARTIFACT_IMAGE:
                global img3
                img3 = Image.open(io.BytesIO(artifact.binary))
                img3.save("C:/Users/do150/Desktop/result/image-img2img.png")
                
    return 1
                

def send_email(sender_email, sender_password, recipient_email, subject, message):
    smtp_server = 'smtp.gmail.com'
    smtp_port = 587

    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = recipient_email
    msg['Subject'] = subject
    msg.attach(MIMEText(message, 'plain'))

    server = smtplib.SMTP(smtp_server, smtp_port)
    server.starttls()
    server.login(sender_email, sender_password)

    server.send_message(msg)
    server.quit()

def timeout_handler():
    raise TimeoutError('Request Timeout')

@app.route('/image', methods=['POST'])
def upload_image():
    timer = threading.Timer(60.0, timeout_handler)
    timer.start()
    try:
        dataset = request.get_json()
        
        key_value = dataset['key']
        print(key_value)
        
        # 로그인
        if key_value == '1':
            iden = dataset['id']
            password = dataset['password']
            
            con = sqlite3.connect("project2.db")
            cur = con.cursor()
            
            cur.execute("SELECT password FROM user WHERE id=?", (iden,))
            result = cur.fetchone()
            
            if result[0] == password:
                response = {'ok_sign': '1'}
                
            else:
                response = {'ok_sign': '0'}
                
            cur.close()
            con.close()
            
            return jsonify(response)
        
        # 회원가입 아이디 확인
        elif key_value == '2':
            con = sqlite3.connect("project2.db")
            
            iden = dataset['id']
            
            query = f"SELECT * FROM user WHERE id = '{iden}'"
            result = con.execute(query).fetchone()
            
            if result is not None:
                response = {'ok_sign': '0'}
                
            else:
                response = {'ok_sign': '1'}
                
            con.close()    
            
            return jsonify(response)
        
        
        # 회원가입
        elif key_value == '3':
            con = sqlite3.connect("project2.db")
            
            iden = dataset['id']
            password = dataset['password']
            nickname = dataset['nickname']
            email = dataset['email']
            
            query = f"INSERT INTO user (id, password, nickname, email) VALUES ('{iden}', '{password}', '{nickname}', '{email}')"
            con.execute(query)
            
            con.commit()
            con.close()
                
            return jsonify({'ok_sign': '1'})
        
        # 정보수정
        elif key_value == '4':
            con = sqlite3.connect("project2.db")
            cur = con.cursor()
            
            iden = dataset['id']
#             change_key = dataset['change_key']

#             if change_key == '1':
#                 password = dataset['password']
#                 c.execute('UPDATE user SET passord = ? WHERE id = ?', (password, iden))
            
#             elif change_key == '2':
#                 nickname = dataset['nickname']
#                 c.execute('UPDATE user SET nickname = ? WHERE id = ?', (nickname, iden))
            
#             elif change_key == '3':
#                 password = dataset['password']
#                 nickname = dataset['nickname']
#                 c.execute('UPDATE user SET passord = ?, nickname = ? WHERE id = ?', (password, nickname, iden))
            
#             else:
#                 print("오류")
                
            password = dataset['password']
            nickname = dataset['nickname']
            cur.execute('UPDATE user SET password = ?, nickname = ? WHERE id = ?', (password, nickname, iden))
            
            con.commit()
            cur.close()
            con.close()
                
            return jsonify({'ok_sign' : '1'}) 
        
        # 갤러리 변환
#         elif key_value == '5':
#             image = dataset['image']
#             sentence = dataset['sentence']
#             img_bytes = base64.b64decode(image)
#             npimg = np.frombuffer(img_bytes, dtype=np.uint8)
#             img_cv = cv2.imdecode(npimg, cv2.IMREAD_COLOR)
            
#             cv2.imwrite('C:/Users/do150/examples/image.png', img_cv)
            
#             do_algorithm()
            
#             algo_image = cv2.imread('C:/Users/do150/Exp/PhotoSketch/Results/image.png')
            
#             stability_algorithm(algo_image, sentence)
            
#             stab_image = cv2.imread("C:/Users/do150/Desktop/result/image-img2img.png")
            
#             _, encoded_stab_image = cv2.imencode('.jpg', stab_image)
#             stab_image_base64 = base64.b64encode(encoded_stab_image).decode('utf-8')

#             return jsonify({'result': stab_image_base64, 'ok_sign' : '1'})
        
        # 갤러리 변환
        elif key_value == '16':
            image = dataset['image']
            point = dataset['point']
            
#             sentence = translate_korean_to_english(sentence)
            point_x = int(float(point[0]))
            point_y = int(float(point[1]))

            img_bytes = base64.b64decode(image)
            npimg = np.frombuffer(img_bytes, dtype=np.uint8)
            img_cv = cv2.imdecode(npimg, cv2.IMREAD_COLOR)
            
            seg_image, seg_image1 = do_seg(img_cv, sam, 'No', point_x, point_y)
            cv2.imwrite("C:/Users/do150/Desktop/soso.png", img_cv)
#             cut_image = remove_grabcut_bg(seg_image)
#             cut_image1 = remove_grabcut_bg(seg_image1)
#             cv2.imwrite("C:/Users/do150/Desktop/soso1.png", cut_image)
#             cv2.imwrite("C:/Users/do150/Desktop/soso2.png", cut_image1)
#             image1 = cv2.cvtColor(cut_image, cv2.COLOR_BGRA2RGBA)
#             image2 = cv2.cvtColor(cut_image1, cv2.COLOR_BGRA2RGBA)
            
            image1 = cv2.cvtColor(seg_image, cv2.COLOR_BGRA2RGBA)
            image2 = cv2.cvtColor(seg_image1, cv2.COLOR_BGRA2RGBA)
            
#             stability_algorithm(img_cv, sentence)

#             stab_image = cv2.imread("C:/Users/do150/Desktop/result/image-img2img.png")
            
#             seg_image2 = do_seg(stab_image, sam, 'No', point_x, point_y)
#             cut_image2 = remove_grabcut_bg(seg_image2)
            
#             cut_image2 = cv2.cvtColor(cut_image2, cv2.COLOR_BGRA2RGBA)
#             image2 = cv2.cvtColor(cut_image2, cv2.COLOR_BGRA2RGBA)
#             cv2.imwrite('C:/Users/do150/Desktop/result4.png', image2)

            _, encoded_stab_image = cv2.imencode('.jpg', image1)
            stab_image_base64 = base64.b64encode(encoded_stab_image).decode('utf-8')
            _, encoded_stab_image1 = cv2.imencode('.jpg', image2)
            stab_image_base641 = base64.b64encode(encoded_stab_image1).decode('utf-8')
            

            return jsonify({'result1': stab_image_base64, 'result2': stab_image_base641, 'ok_sign' : '1'})
        
        # 스케치 변환
        elif key_value == '6':
            sentence = dataset['sentence']
            
            sentence = translate_korean_to_english(sentence)
            
            text2image(sentence)
            
            stab_image = cv2.imread("C:/Users/do150/Desktop/result/image-img2img.png")

            _, encoded_stab_image = cv2.imencode('.jpg', stab_image)
            stab_image_base64 = base64.b64encode(encoded_stab_image).decode('utf-8')

            return jsonify({'result': stab_image_base64, 'ok_sign' : '1'})
        
        # 스토리 만들기
        elif key_value == '7':
            sentence = dataset['sentence']
            page = dataset['page']
            character = dataset['character']
            
            answer = openai_algo(sentence, page, character)

            last_answer = split_answer(answer)

            return jsonify({'answer': last_answer, 'ok_sign' : '1'})

        
        # 스토리 저장 여러 데이터 다 받아서 정리해야됨
        elif key_value == '8':
            iden = dataset['id']
            story = dataset['story']
            image = dataset['image']
            title = dataset['title']

            save_result_story(iden, story, image, title)
                       
            return jsonify({'ok_sign' : '1'})
            
        # 동화책 만들기
        elif key_value == '9':
            iden = dataset['id']
            category = dataset['cate']
            
            
            print(iden, category)
            
            con = sqlite3.connect("project2.db")
            cur = con.cursor()

            cur.execute("SELECT o.image, o.sentence FROM object o JOIN objecttable ot ON o.imageid = ot.imageid WHERE ot.id = ? AND o.category = ?", (iden, category))
            result = cur.fetchall()
            print(result)
            arr1 = []
            arr2 = []
                    
            for i, j in result:
                img = cv2.imread(i)
                jpg_img = cv2.imencode('.jpg', img)
                b64_string = base64.b64encode(jpg_img[1]).decode('utf-8')
                arr1.append(b64_string)
                arr2.append(j)
                
            cur.close()
            con.close()
            
            return jsonify({'image': arr1, 'sentence': arr2, 'ok_sign': '1'})
            
        # 스토리 리스트
        elif key_value == '10':
            # page = dataset['page']
            iden = dataset['id']
            
            con = sqlite3.connect("project2.db")
            cur = con.cursor()

            query1 = f"SELECT story.storyid FROM storytable JOIN story ON storytable.storyid=story.storyid WHERE storytable.id='{iden}'"
            query2 = f"SELECT story.title FROM storytable JOIN story ON storytable.storyid=story.storyid WHERE storytable.id='{iden}'"

            cur.execute(query1)
            result1 = cur.fetchall()
            cur.execute(query2)
            result2 = cur.fetchall()
            
            a = []
            b = []
            
            for i in range(len(result1)):
                ad = result1[i][0]
                bd = result2[i][0]
                a.append(ad)
                b.append(bd)
            
            cur.close()
            con.close()
    
            return jsonify({'title': a, 'storyid': b, 'ok_sign': '1'})
        
        # 스토리 열람
        elif key_value == '11':
            iden = dataset['id']
            story_id = dataset['storyid']
            page = dataset['page']
            
            con = sqlite3.connect("project2.db")
            cur = con.cursor()
            cur.execute('PRAGMA foreign_keys = ON')
            
            cur.execute("SELECT story FROM story WHERE storyid=? AND storyid IN (SELECT storyid FROM storytable WHERE id=?)", (story_id, iden))
            story_text = cur.fetchone()[0]

            image_lines = []
            story_lines = []

            with open(story_text, "r") as f:
                lines = f.readlines()
                image_lines = [line.strip() for i, line in enumerate(lines) if i % 2 == 0]
                story_lines = [line.strip() for i, line in enumerate(lines) if i % 2 == 1]

            cur.close()
            con.close()

            length = len(story_lines)

            if length < page:
                return jsonify({'ok_sign': '0'})
            
            else:
                image_send = cv2.imread(image_lines[page])
                jpg_img = cv2.imencode('.jpg', image_send)
                b64_string = base64.b64encode(jpg_img[1]).decode('utf-8')
                
                return jsonify({'image': b64_string, 'story': story_lines[page], 'ok_sign': '1'})
        
        # 초기화면 닉네임 표시
        elif key_value == '12':
            iden = dataset['id']
            
            con = sqlite3.connect("project2.db")
            cur = con.cursor()
            
            cur.execute("SELECT nickname FROM user WHERE id=?", (iden,))
            result = cur.fetchone()
            
            cur.close()
            con.close()
            
            return jsonify({'nickname': result[0], 'ok_sign' : '1'})
        
        # 이미지 저장
        elif key_value == '13':
            iden = dataset['id']
            category = dataset['category']
            image = dataset['image']
            share = dataset['share']        
            sentence = dataset['sentence']
            
            img_bytes = base64.b64decode(image)
            npimg = np.frombuffer(img_bytes, dtype=np.uint8)
            img_cv = cv2.imdecode(npimg, cv2.IMREAD_COLOR)
            
            save_result_image(img_cv, iden, category, share, sentence)
            
            return jsonify({'ok_sign' : '1'})
        
        # 이메일 확인
        elif key_value == '14':
            email = dataset['email']
            print(email)
            con = sqlite3.connect("project2.db")
            
            query = f"SELECT * FROM user WHERE email = '{email}'"
            result = con.execute(query).fetchone()
            print(result)
            if result is None:
                response = {'ok_sign': '0'}
                
            else:
                sender_email = 'do15071111@gmail.com'  # 발신자 이메일 주소
                sender_password = 'znigmkalajiafxzc'  # 발신자 이메일 비밀번호
                recipient_email = email
                subject = '아이디 비밀번호 알림'
                message = '아이디 : ' + result[0] + '\n' + '비밀번호 : ' + result[1]

                send_email(sender_email, sender_password, recipient_email, subject, message)

                response = {'ok_sign': '1'}
                
            con.close()    
            
            return jsonify(response)
        
        # 그림 좌표 전송
        elif key_value == '15':
            image = dataset['image']
            point_x = dataset['x']
            point_y = dataset['y']
            
            point_x = int(float(point_x))
            point_y = int(float(point_y))
            
            img_bytes = base64.b64decode(image)
            npimg = np.frombuffer(img_bytes, dtype=np.uint8)
            img_cv = cv2.imdecode(npimg, cv2.IMREAD_COLOR)
            
            seg_image2 = do_seg(img_cv, sam, 'No', point_x, point_y)
            cut_image2 = remove_grabcut_bg(seg_image2)
            
            cut_image2 = cv2.cvtColor(cut_image2, cv2.COLOR_BGRA2RGBA)
            image2 = cv2.cvtColor(cut_image2, cv2.COLOR_BGRA2RGBA)
            cv2.imwrite('C:/Users/do150/Desktop/result4.png', image2)

            _, encoded_stab_image = cv2.imencode('.jpg', stab_image)
            stab_image_base64 = base64.b64encode(encoded_stab_image).decode('utf-8')

            return jsonify({'result': stab_image_base64, 'ok_sign' : '1'})
        
        elif key_value == '5':
            image = dataset['image']
            sentence = dataset['sentence']
            
            sentence = translate_korean_to_english(sentence)

            img_bytes = base64.b64decode(image)
            npimg = np.frombuffer(img_bytes, dtype=np.uint8)
            img_cv = cv2.imdecode(npimg, cv2.IMREAD_COLOR)
            
            stability_algorithm(img_cv, sentence)
            print('ok')

            stab_image = cv2.imread("C:/Users/do150/Desktop/result/image-img2img.png")
            
            _, encoded_stab_image = cv2.imencode('.jpg', stab_image)
            stab_image_base64 = base64.b64encode(encoded_stab_image).decode('utf-8')

            return jsonify({'result': stab_image_base64, 'ok_sign' : '1'})
        
        #공유 가져오기 기능
        elif key_value == '17':
            cate = dataset['category']
            print(cate)
            # 데이터베이스 연결
            con = sqlite3.connect("project2.db")
            cur = con.cursor()
            
#             if cate == 's':
#                 con = sqlite3.connect("project1.db")
#                 cur = con.cursor()

#                 cur.execute("SELECT storyid FROM storytable WHERE share = '1'")
#                 image_ids = cur.fetchall()

#                 for image_id in image_ids:
#                     cur.execute("SELECT image FROM story WHERE imageid = ?", (image_id[0],))
#                     result = cur.fetchone()
#                     if result:
#                         image_data = result[0]
#                         print("이미지 데이터:", image_data)
                
            if cate == 'o':
                con = sqlite3.connect("project2.db")
                cur = con.cursor()

                cur.execute("SELECT imageid FROM objecttable WHERE share = '1'")
                image_ids = cur.fetchall()
                print(image_ids)
                arr = []
                arr1 = []

                for image_id in image_ids:
                    cur.execute("SELECT image, sentence FROM object WHERE imageid = ? AND category = 'o'", (image_id[0],))
                    result = cur.fetchone()
                    
                    if result is None:
                        continue
                    
                    image_data = result[0]
                    img = cv2.imread(image_data)
                    jpg_img = cv2.imencode('.jpg', img)
                    b64_string = base64.b64encode(jpg_img[1]).decode('utf-8')
                    arr.append(b64_string)
                    image_sentence = result[1]
                    arr1.append(image_sentence)
                     
                cur.close()
                con.close()
                
                return jsonify({'result': arr, 'sentence': arr1, 'ok_sign' : '1'})
                        
            elif cate == 'b':
                con = sqlite3.connect("project2.db")
                cur = con.cursor()

                cur.execute("SELECT imageid FROM objecttable WHERE share = '1'")
                image_ids = cur.fetchall()
                print(image_ids)
                arr = []
                arr1 = []

                for image_id in image_ids:
                    cur.execute("SELECT image, sentence FROM object WHERE imageid = ? AND category = 'b'", (image_id[0],))
                    result = cur.fetchone()
                    
                    if result is None:
                        continue
                    
                    image_data = result[0]
                    img = cv2.imread(image_data)
                    jpg_img = cv2.imencode('.jpg', img)
                    b64_string = base64.b64encode(jpg_img[1]).decode('utf-8')
                    arr.append(b64_string)
                    image_sentence = result[1]
                    arr1.append(image_sentence)
                        
                cur.close()
                con.close()
                        
                return jsonify({'result': arr, 'sentence': arr1, 'ok_sign' : '1'})
            
            else:
                cur.close()
                con.close()
                return jsonify({'ok_sign' : '0'})
        
        # 공유 다운로드 기능
        elif key_value == '18':
            iden = dataset['id']
            category = dataset['category']
            image = dataset['image']
            sentence = dataset['sentence']
            
            img_bytes = base64.b64decode(image)
            npimg = np.frombuffer(img_bytes, dtype=np.uint8)
            img_cv = cv2.imdecode(npimg, cv2.IMREAD_COLOR)
            
            save_result_image(img_cv, iden, category, '0', sentence)
            
            return jsonify({'ok_sign' : '1'})
        
        elif key_value == '19':
            image = dataset['image']
            sentence = dataset['sentence']
            style = dataset['style']
            print(sentence)
            combined_string = sentence + style
            print(combined_string)
            sent = translate_korean_to_english(combined_string)

            img_bytes = base64.b64decode(image)
            npimg = np.frombuffer(img_bytes, dtype=np.uint8)
            img_cv = cv2.imdecode(npimg, cv2.IMREAD_COLOR)
            print('1')
            stability_algorithm(img_cv, sent)
            print('ok')

            stab_image = cv2.imread("C:/Users/do150/Desktop/result/image-img2img.png")
            
            _, encoded_stab_image = cv2.imencode('.jpg', stab_image)
            stab_image_base64 = base64.b64encode(encoded_stab_image).decode('utf-8')
            
            return jsonify({'result': stab_image_base64, 'ok_sign' : '1'})
 
        
    except TimeoutError:
        return 'Request Timeout', 408
    except Exception as e:
        return str(e), 500
    finally:
        # 타임아웃 시그널 해제
        timer.cancel()

def split_answer(answer):
    trans_answer = answer.split("\n")
    trans_answer = [i for i in trans_answer if i != '']
    
    return trans_answer

def save_result_image(image, iden, category, share, sentence):
    with open("C:/Users/do150/Desktop/count.txt", "r") as f:
        count = f.read()
    
    con = sqlite3.connect("project2.db")
            
    result_image_file_path = 'C:/Users/do150/Desktop/result_images/image'
    result_image_file_name = result_image_file_path + str(count) + '.png'
    
    image_query = f"INSERT INTO object (image, category, imageid, sentence) VALUES ('{result_image_file_name}', '{category}', '{count}', '{sentence}')"
    con.execute(image_query)
    
    plus_query = f"INSERT INTO objecttable (imageid, id, share) VALUES ('{count}', '{iden}', '{share}')"
    con.execute(plus_query)
    
    con.commit()
    con.close()
    
    image1 = cv2.cvtColor(image, cv2.COLOR_BGRA2RGBA)
    image_ch = cv2.cvtColor(image1, cv2.COLOR_BGRA2RGBA)
    cv2.imwrite(result_image_file_name, image_ch)
            
    num_int = int(count) + 1
            
    with open("C:/Users/do150/Desktop/count.txt", "w") as f:
        f.write(str(num_int))
    
    return 1

# 나중에 스토리 전체 저장 파일로 바꿀것
def save_result_story(iden, story, image, title):
    with open("C:/Users/do150/Desktop/story_count.txt", "r") as f:
        count = f.read()
        
    con = sqlite3.connect("project2.db")

    middle_tale_file_path = 'C:/Users/do150/Desktop/result_tale_image/img'
    result_tale_file_path = 'C:/Users/do150/Desktop/result_tale/tale'
    result_tale_file_name = result_tale_file_path + str(count) + '.txt'

    result_image_path = []
    for images in image:
        with open("C:/Users/do150/Desktop/story_image_count.txt", "r") as f:
            count_image = f.read()

        img_bytes = base64.b64decode(images)
        npimg = np.frombuffer(img_bytes, dtype=np.uint8)
        img_cv = cv2.imdecode(npimg, cv2.IMREAD_COLOR)

        middle_tale_image_name = middle_tale_file_path + str(count_image) + '.png'

        cv2.imwrite(middle_tale_image_name, img_cv)
        result_image_path.append(middle_tale_image_name)
        
        image_int = int(count_image) + 1
        
        with open("C:/Users/do150/Desktop/story_image_count.txt", "w") as f:
            f.write(str(image_int))
    
    with open(result_tale_file_name, "w") as re:
        for i in range(len(result_image_path)):
            re.write(result_image_path[i] + '\n')
            re.write(story[i] + '\n')
    
    image_query = f"INSERT INTO story (story, title, storyid) VALUES ('{result_tale_file_name}', '{title}', '{count}')"
    con.execute(image_query)
    
    plus_query = f"INSERT INTO storytable (id, storyid) VALUES ('{iden}', '{count}')"
    con.execute(plus_query)

    con.commit()
    con.close()
    
    num_int = int(count) + 1

    with open("C:/Users/do150/Desktop/story_count.txt", "w") as f:
        f.write(str(num_int))
    
    return 1

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.102:5000
Press CTRL+C to quit
198.235.24.58 - - [20/Sep/2023 15:12:28] "GET / HTTP/1.0" 404 -
